In [1]:
sc.master

u'local[*]'

In [91]:
from __future__ import division, print_function, unicode_literals
import jieba

In [4]:
sc.textFile("news.txt").count()

6

In [37]:
def tokenize(line):
   return (list(jieba.cut(line)), )

tokenize("我很愛你")

#("Hi I heard about Spark".split(" "),)
#(list(jieba.cut("我很愛你")), )

([u'\u6211', u'\u5f88', u'\u611b', u'\u4f60'],)

In [38]:
from pyspark.ml.feature import Word2Vec

documentRDD = sc.textFile("news.txt") \
    .map(tokenize)

In [39]:
documentRDD.take(4)

[([u'\u75be\u75c5',
   u'\u7ba1\u5236',
   u'\u7f72',
   u'\u4e0a\u5348',
   u'11',
   u'\u6642',
   u'\u6b63\u5f0f',
   u'\u516c\u5e03',
   u'\u570b\u5167',
   u'\u4eca\u5e74',
   u'\u9996',
   u'\u8d77',
   u'\u5883\u5916',
   u'\u79fb\u5165',
   u'H7N9',
   u'\u78ba\u5b9a',
   u'\u75c5\u4f8b',
   u'\uff0c',
   u'\u662f',
   u'\u6b77\u5e74',
   u'\u7b2c',
   u'5',
   u'\u4f8b',
   u'\uff0c',
   u'\u70ba',
   u'\u5bb6\u4f4f',
   u'\u9ad8\u96c4',
   u'\u7684',
   u'\u672c\u570b\u7c4d',
   u'69',
   u'\u6b72',
   u'\u7537\u6027',
   u'\uff0c',
   u'\u66fe',
   u'\u6709',
   u'\u4e2d\u570b',
   u'\u5ee3\u6771',
   u'\u7701\u967d\u6c5f\u5e02',
   u'\u6d3b\u52d5\u53f2',
   u'\uff0c',
   u'\u7121\u79bd',
   u'\u9ce5',
   u'\u6216',
   u'\u7591\u4f3c\u75c5\u4f8b',
   u'\u63a5\u89f8\u53f2',
   u'\uff0c',
   u'\u65e5\u524d',
   u'\u65bc',
   u'\u5ee3\u6771\u7701',
   u'\u767c\u75c5',
   u'\uff0c',
   u'\u5165\u5883',
   u'\u6642',
   u'\u63a5\u53d7',
   u'\u767c\u71d2',
   u'\u7be9\u6aa2',
   

In [40]:
documentRDD.count()

6

In [41]:
docDF = documentRDD.toDF(["text"])
docDF.printSchema()

root
 |-- text: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [44]:
docDF.show()

+--------------------+
|                text|
+--------------------+
|[疾病, 管制, 署, 上午, 1...|
|[針對, 此個案, ，, 疾管署,...|
|[不過, 該, 男, 於, 1, ...|
|[羅, 一鈞, 說明, ，, 經衛...|
|[根據, 疾管署, 資料, ，, ...|
|[目前, 疾管署, 針對, 人類,...|
+--------------------+



In [43]:
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(docDF)

In [45]:
result = model.transform(docDF)
result.show()

+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|[疾病, 管制, 署, 上午, 1...|[0.01182220279095...|
|[針對, 此個案, ，, 疾管署,...|[0.04050601742135...|
|[不過, 該, 男, 於, 1, ...|[0.01128114456514...|
|[羅, 一鈞, 說明, ，, 經衛...|[0.04870357851842...|
|[根據, 疾管署, 資料, ，, ...|[0.02757538204443...|
|[目前, 疾管署, 針對, 人類,...|[0.02621794550213...|
+--------------------+--------------------+



In [49]:
text, vector = result.take(1)[0]
vector

DenseVector([0.0118, 0.0092, 0.0314])

In [54]:
from pyspark.mllib.feature import Word2Vec
documentRDD = sc.textFile("news.txt") \
    .map(lambda l: list(jieba.cut(l)))

word2Vec = Word2Vec()
model = word2Vec.fit(documentRDD)

In [108]:
word_index_rdd = documentRDD.flatMap(lambda words: words) \
    .distinct() \
    .zipWithIndex() \
    .map(lambda x: (x[0], str(x[1] + 1)))

vocabs = word_index_rdd.collectAsMap()
indices = word_index_rdd \
    .map(lambda x: (x[1], x[0])).collectAsMap()

In [95]:
vocabs['管制']
vocabs.get('ddd', '0')

u'0'

In [109]:
indices['2']

u'\u7f85'

In [105]:
b_vocs = sc.broadcast(vocabs)
wordIdsRDD = documentRDD.map(lambda words: map(lambda w: b_vocs.value.get(w, '0'), words))

word2Vec = Word2Vec()
word2Vec.setMinCount(0)
word2Vec.setVectorSize(3)
model = word2Vec.fit(wordIdsRDD)

In [106]:
model.getVectors()

{u'205': [-0.15178613, 0.12007163, 0.07854693], u'45': [0.10725516, 0.09691736, 0.11646473], u'98': [0.09231491, 0.035964116, -0.11300027], u'113': [0.08265991, 0.009741492, -0.15570492], u'34': [0.14863132, -0.14306584, 0.114977546], u'67': [-0.15360302, 0.076470494, -0.027055278], u'169': [-0.1213683, 0.15860415, 0.12155242], u'120': [0.081432804, 0.091122806, 0.04480915], u'193': [-0.040647447, -0.03534899, 0.07582277], u'153': [0.0509461, 0.05633232, -0.13121876], u'93': [0.065410785, 0.14019348, -0.115169115], u'158': [0.05797889, 0.001368305, -0.006482792], u'142': [-0.11589702, -0.045742806, -0.15997443], u'182': [-0.1561173, -0.10144212, 0.15406637], u'147': [0.0047371876, 0.021837052, 0.06620025], u'12': [-0.122661754, -0.08072154, 0.035172842], u'223': [0.04984197, -0.15509775, -0.14744027], u'66': [0.0049799085, -0.14733332, 0.07347997], u'216': [-0.072899036, -0.1306169, 0.11957706], u'89': [0.092604436, 0.118035674, 0.051454462], u'51': [0.08020468, 0.10500557, 0.039271902

In [111]:
target = vocabs['疾病']
synonyms = model.findSynonyms(target, 5)

for index, cosine_distance in synonyms:
    word = indices[index]
    print("{}: {}".format(word, cosine_distance))

就: 0.998995535962
多: 0.997564701906
廣東省: 0.992267550375
呼吸: 0.987810078002
醫院: 0.953878190235
